In [7]:
#  Step 1: Install required libraries
!pip install geopandas folium plotly

#  Step 2: Import necessary libraries
import pandas as pd
import folium
import seaborn as sns
import matplotlib.pyplot as plt
from folium.plugins import MarkerCluster
import plotly.express as px

#  Step 3: Load your dataset
df = pd.read_csv('/content/drive/MyDrive/Cognifyz/Dataset .csv')

# Display the first few rows
df.head(3)

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270


In [ ]:
# Interactive Restaurant Data Visualization

#  Step 1: Clean column names
df.columns = df.columns.str.strip()

#  Step 2: Drop rows without coordinates
df = df.dropna(subset=['Latitude', 'Longitude'])

#  Step 3: Grouping by city or locality
group_col = 'City' if 'City' in df.columns else 'Locality'

# Restaurant count per location
group_count = df.groupby(group_col).size().reset_index(name='restaurant_count').sort_values('restaurant_count', ascending=False)

#  Step 4: Aggregated details (avg rating, price range, common cuisine)
agg_df = df.groupby(group_col).agg({
    'Aggregate rating': 'mean',
    'Price range': 'mean',
    'Cuisines': lambda x: x.mode()[0] if not x.mode().empty else 'Unknown'
}).reset_index().sort_values('Aggregate rating', ascending=False)

#  Step 5: Bar chart – Top 10 by restaurant count
fig1 = px.bar(group_count.head(10), x=group_col, y='restaurant_count', title=f'Top 10 {group_col}s by Restaurant Count')
fig1.show()

#  Step 6: Bar chart – Top 10 by average rating
fig2 = px.bar(agg_df.head(10), x=group_col, y='Aggregate rating', title=f'Top 10 {group_col}s by Average Rating')
fig2.show()

#  Step 7: Scatter Plot – Mapbox style world rating map
fig3 = px.scatter_mapbox(df,
                         lat="Latitude", lon="Longitude",
                         color="Aggregate rating",
                         hover_name="Restaurant Name",
                         hover_data=["Cuisines", "Address"],
                         mapbox_style="carto-positron",
                         zoom=1,
                         title="Global Restaurant Ratings Map")
fig3.show()

#  Step 8: Folium interactive map – Zoomable marker cluster
restaurant_map = folium.Map(location=[20, 0], zoom_start=2)
marker_cluster = MarkerCluster().add_to(restaurant_map)

# Function to assign marker color based on rating
def get_marker_color(rating):
    try:
        rating = float(rating)
        if rating >= 4.5:
            return 'darkgreen'
        elif rating >= 4.0:
            return 'green'
        elif rating >= 3.0:
            return 'orange'
        elif rating > 0:
            return 'red'
        else:
            return 'gray'
    except:
        return 'gray'

for _, row in df.iterrows():
    popup_text = f"<b>{row.get('Restaurant Name', 'Unknown')}</b><br>Rating: {row.get('Aggregate rating', 'N/A')}<br>Cuisines: {row.get('Cuisines', 'Not specified')}<br>Address: {row.get('Address', 'No address')}"
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=folium.Popup(popup_text, max_width=300),
        icon=folium.Icon(color=get_marker_color(row.get('Aggregate rating')), icon='cutlery', prefix='fa')
    ).add_to(marker_cluster)

restaurant_map


Output hidden; open in https://colab.research.google.com to view.